In [ ]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)
%cd '/content/drive/My Drive/'

Mounted at /content/drive
/content/drive/My Drive


In [ ]:
!pip install leidenalg igraph

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 22.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 49.4 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import scipy.io as sio
import numpy as np
import igraph as ig
import leidenalg
from itertools import combinations
mat=sio.loadmat('A.mat')['A'] #Import the multiplex network of sparse kernel matrices
mat

In [ ]:
def Divisive_Clustering(X):
    """
    Divisive Kernel Clustering using the Leiden algorithm.

    Arguments:
    X : 3D numpy array of shape (N, N, L), where N is the number of nodes,
        and L is the number of layers (each representing a weighted adjacency matrix).

    Returns:
    Opt_rank : The optimal number of clusters to extract
    M : Community assignment of nodes
    Q : Modularity value of the partition
    aggregated_vs : Co-membership matrix aggregated across layers
    """
    # Validate input dimensions
    if len(X.shape) != 3 or X.shape[0] != X.shape[1]:
        raise ValueError("Input X must be a 3D numpy array with shape (N, N, L).")

    N, _, L = X.shape  # Number of nodes and layers
    Vs = []  # To store co-membership matrices from each layer
    combos_s = list(combinations(range(N), 2))  # Generate pairs of nodes

    for i in range(L):
        v = []  # Co-membership vector for the current layer
        net = X[:, :, i]  # Extract the i-th layer as a NumPy array

        # Create an igraph directed graph for each layer
        G = ig.Graph.Weighted_Adjacency(net.tolist(), mode="directed", attr="weight")
        partition = leidenalg.find_partition(G, leidenalg.ModularityVertexPartition, weights="weight")

        M = partition.membership

        # Compute co-membership matrix for the current layer
        co_matrix = np.zeros((N, N))
        for ii in range(len(combos_s)):
            m1 = M[combos_s[ii][0]]
            m2 = M[combos_s[ii][1]]
            if m1 == m2:
                co_matrix[combos_s[ii][0], combos_s[ii][1]] = net[combos_s[ii][0], combos_s[ii][1]]+net[combos_s[ii][1], combos_s[ii][0]]
                co_matrix[combos_s[ii][1], combos_s[ii][0]] = net[combos_s[ii][0], combos_s[ii][1]]+net[combos_s[ii][1], combos_s[ii][0]]  # Ensure symmetry

        Vs.append(co_matrix)

    # Convert the list of co-membership matrices to a 3D array
    Vs = np.stack(Vs, axis=2)

    # Aggregate the co-membership matrices by summing across layers
    aggregated_vs = np.sum(Vs, axis=2)

    # Create a graph for the aggregated matrix (using undirected mode)
    # Aggregated co-membership matrix should be treated as undirected
    G = ig.Graph.Weighted_Adjacency(aggregated_vs.tolist(), mode="undirected", attr="weight")
    partition = leidenalg.find_partition(G, leidenalg.ModularityVertexPartition, weights="weight")

    Q = partition.q  # Modularity score
    M = partition.membership

    # Return the optimal number of clusters and other results
    Opt_rank = max(M) + 1
    return Opt_rank, M, aggregated_vs



In [ ]:
Opt_rank, M, aggregated_vs= Divisive_Clustering(mat)
print("Optimal number of clusters:", Opt_rank)
print("Community assignments:", M)
print("Aggregated co-membership matrix (Vs):\n", aggregated_vs)


Optimal number of clusters: 2
Community assignments: [0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0]
Aggregated co-membership matrix (Vs):
 [[0.00000000e+00 7.10219191e-01 7.89388655e+00 6.95220453e+00
  2.16824745e-03 1.28604033e+01 3.50318767e+00 5.97815796e+00
  2.42126024e+00 1.18204402e+01 5.63345042e+00 1.04555722e+00
  2.79647442e+00 2.59699038e+00 2.13370096e+00 4.42029426e+00
  1.83659632e+00 3.78502118e+00 1.99699305e+00 4.89363295e+00]
 [7.10219191e-01 0.00000000e+00 2.09605019e+00 8.42162801e+00
  1.63609381e+01 0.00000000e+00 6.31767750e+00 1.45515508e+01
  4.25371801e+00 1.02743072e+00 8.85093166e+00 1.18217025e+01
  1.51312758e+01 1.26759060e+01 8.07503093e+00 7.56076527e+00
  5.70748369e+00 7.51798728e+00 1.11580956e+01 1.71149857e+00]
 [7.89388655e+00 2.09605019e+00 0.00000000e+00 4.58622852e+00
  2.83960220e+00 7.42457004e+00 5.06242763e+00 6.66925301e+00
  1.89634214e+00 1.14850117e+01 3.32306424e+00 3.58386543e+00
  4.49375842e+00 4.04142329e+00 1.63188

In [ ]:
# Example usage
# Assuming `mat` is your 3D numpy array (shape: N x N x L)
result_tree = recursive_Divisive_Clustering(mat, max_depth=4,current_depth=0)
result_tree